In [ ]:
import os
import ast
import json
import pandas as pd

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime

In [ ]:
#############################################
### TRABAJAMOS SOBRE EL ARCHIVO DE JUEGOS ###
#############################################

datos = []            # Definimos/limpiamos una lista que servira como nexo entre los datos y el dataframe.
with open('output_steam_games.json') as archivo:            # Abrimos el archivo json.
    for linea in archivo:
        datos.append(json.loads(linea))         # Agregamos cada linea del json a la lista ya creada.

dfGames = pd.DataFrame(datos)          # Creamos un dataframe con los datos de la lista.
dfGames = dfGames.dropna(axis=0, how='all')           # Eliminamos del dataframe las filas en las que todos sus campos son nulos.

dfGames = dfGames.drop(['publisher', 'title', 'url', 'specs', 'price', 'reviews_url', 'price', 'early_access', 'developer'], axis=1)          # Eliminamos las columnos que no seran de ayuda en el analisis.

dfGames = dfGames.dropna(subset=['release_date'])        # Eliminanos los datos nan de la columna de fechas.

for fila in dfGames['release_date']:
    try:
        dfGames.loc[dfGames['release_date'] == fila, 'nueva_fecha'] = datetime.strptime(fila, "%Y-%m-%d").year         # Guardamos en una nueva columna, las fechas con formato valido.
    except ValueError:
       dfGames.loc[dfGames['release_date'] == fila, 'nueva_fecha'] = None         # Si la fecha no es valida, guardamos un nan en su lugar.
       

dfGames = dfGames.drop(['release_date'], axis = 1)            # Eliminamos la columna que ya no sera de ayuda.
dfGames = dfGames.rename(columns = {'nueva_fecha': 'release_date'})         # Renombramos la nueva columna con el nombre que ya nos hemos familiarizado.
dfGames = dfGames.dropna(subset = ['release_date'])        #Eliminanos los datos nan de la columna de fechas.
dfGames = dfGames.explode('genres')

In [ ]:
dfGames.head()

In [ ]:
##############################################
### TRABAJAMOS SOBRE EL ARCHIVO DE REVIEWS ###
##############################################

datos=[]            # Definimos/limpiamos una lista que servira como nexo entre los datos y el dataframe.

with open('australian_user_reviews.json') as archivo:           # Abrimos el archivo json.
    for lineas in archivo:
        linea = ast.literal_eval(lineas)            # Ayuda a manejar el "json" sin que el formato traiga problemas.
        datos.append(linea)         # Agregamos cada linea del "json" a la lista ya creada.

dfReviews = pd.DataFrame(datos)         # Creamos un dataframe con los datos de la lista.
dfReviews = dfReviews.drop(['user_url'], axis = 1)            # Eliminamos las columnos que no seran de ayuda en el analisis.


dfReviews = dfReviews.drop_duplicates('user_id')            # Eliminamos los usuarios que esten duplicados.
dfReviews = dfReviews.explode('reviews')          # Se desanidan los reviews, trasladando cada una a una fila diferente.

dfReviews.dropna()          # Eliminamos los registros nan.
dfReviews = dfReviews.dropna(subset = ['reviews'])            # Eliminamos las filas que no tengan reviews.

for key in dfReviews['reviews'].iloc[0].keys():         
    dfReviews[key] = dfReviews['reviews'].apply(lambda x: x[key])         # Toma cada "llave/valor" correspondiente a las reviews y arma columnas aparte para tener los datos ordenados y accesibles.
    
dfReviews = dfReviews.drop(['reviews', 'funny', 'last_edited', 'helpful'], axis = 1)         # Eliminamos la columna REVIEWS, la cual ya fue desanidada y otras que no utilizaremos.

In [ ]:
dfReviews.head()

In [ ]:
############################################
### TRABAJAMOS SOBRE EL ARCHIVO DE ITEMS ###
############################################

datos=[]            # Definimos/limpiamos una lista que servira como nexo entre los datos y el dataframe.

with open('australian_users_items.json') as archivo:            # Abrimos el archivo json.
    for lineas in archivo:
        linea = ast.literal_eval(lineas)         # Ayuda a manejar el "json" sin que el formato traiga problemas.
        datos.append(linea)         # Agregamos cada linea del "json" a la lista ya creada.

dfItems = pd.DataFrame(datos)         # Creamos un dataframe con los datos de la lista.
dfItems = dfItems.drop(['user_url', 'steam_id'], axis=1)            # Eliminamos las columnos que no seran de ayuda en el analisis.


dfItems = dfItems.drop_duplicates('user_id')            # Eliminamos los usuarios que esten duplicados.
dfItems = dfItems.drop(dfItems[(dfItems['items_count'] == 0)].index)            # Eliminamos los usuarios que no tengan juegos.
dfItems = dfItems.explode('items')          # Se desanidan los juegos, trasladando cada uno a una fila diferente.

for key in dfItems['items'].iloc[0].keys():         
    dfItems[key] = dfItems['items'].apply(lambda x: x[key])         # Toma cada "llave/valor" correspondiente a los juegos y arma columnas aparte para tener los datos ordenados y accesibles.
    
dfItems = dfItems.drop(['items', 'items_count', 'playtime_2weeks'], axis = 1)         # Eliminamos la columna ITEMS, la cual ya fue desanidada y otras que no utilizaremos.

In [ ]:
dfItems.head()

In [ ]:
#######################
### PRIMER CONSULTA ###
#######################

dfAux = dfItems.groupby(by = 'item_id').playtime_forever.sum()            # Agrupamos los juegos, obteniendo las horas totales de juego.
dfResultadoU = pd.merge(dfGames, dfAux, how = 'inner', left_on='id', right_on = 'item_id')         # Agregamos al dataframe JUEGOS, el total de horas jugadas en c/u.
dfResultadoU = dfResultadoU.dropna(subset=['genres'])        #Eliminanos los datos nan de la columna de fechas.

dfResultadoU = dfResultadoU.groupby(['genres' , 'release_date'])['playtime_forever'].sum().reset_index()           # Agrupamos los datos por genero y año, sumando las horas.
dfResultadoU['max'] = dfResultadoU.groupby('genres')['playtime_forever'].transform(max)           # Detectamos la mayor cantidad de horas y las agregamos a una columna auxiliar.
dfResultadoU = dfResultadoU[dfResultadoU['playtime_forever'] == dfResultadoU['max']]         # Filtramos el dataframe para que traiga solo los generos y el año con la mayor cantidad de horas.
dfResultadoU['release_date'] = dfResultadoU['release_date'].astype('int')         # Converimos el año a formato numerico.

### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok = True)
dfResultadoU.to_csv('Resultados/PConsulta.csv')

In [ ]:
dfResultadoU.head()

In [ ]:
########################
### SEGUNDA CONSULTA ###
########################

dfResultadoD = pd.merge(dfGames, dfReviews, how = 'inner', left_on='id', right_on='item_id')            # Creamos un nuevo dataframe, agrupando los juegos y las reviews.
dfResultadoD = pd.merge(dfResultadoD, dfItems, how='inner', left_on=['user_id', 'item_id'], right_on=['user_id', 'item_id'])            # Agregamos al mismo dataframe, el tiempo juegado.
dfResultadoD = dfResultadoD.drop(['tags', 'release_date',  'item_id', 'recommend', 'review', 'item_name'], axis = 1)          # Eliminamos las columnas que no seras utilizadas.

for fila in dfResultadoD['posted']:
    try:
        dfResultadoD.loc[dfResultadoD['posted'] == fila, 'nueva_fecha'] = datetime.strptime(fila, 'Posted %B %d, %Y.').year         # Guardamos en una nueva columna, las fechas con formato valido (poseen anio).
    except ValueError:
       dfResultadoD.loc[dfResultadoD['posted'] == fila, 'nueva_fecha'] = None         # Si la fecha no es valida (no posee anio), guardamos un nan en su lugar.

dfResultadoD = dfResultadoD.dropna(subset=['nueva_fecha'])        #Eliminanos los datos nan de la columna de fechas.
dfResultadoD = dfResultadoD.groupby(['user_id', 'nueva_fecha', 'genres'])['playtime_forever'].sum().reset_index()           # Agrupamos los datos por usuario, anio y genero, sumando las horas.
dfResultadoD['max'] = dfResultadoD.groupby(['user_id', 'genres'])['playtime_forever'].transform(sum)           # Sumamos las horas totales por genero de cada jugador y las agregamos a una columna auxiliar.
dfResultadoD = dfResultadoD[dfResultadoD.groupby("genres")["max"].transform(max) == dfResultadoD["max"]]            # Filtramos los datos por la maxima cantidad de horas, dejando solo los que necesitamos para la consulta.
dfResultadoD['nueva_fecha'] = dfResultadoD['nueva_fecha'].astype('int')         # Converimos el año a formato numero.


### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok = True)
dfResultadoD.to_csv('Resultados/SConsulta.csv')

In [ ]:
dfResultadoD.head()

In [ ]:
##########################
### 3ra y 4ta CONSULTA ###
##########################

dfResultadoT = dfReviews

for fila in dfResultadoT['posted']:
    try:
        dfResultadoT.loc[dfResultadoT['posted'] == fila, 'nueva_fecha'] = datetime.strptime(fila, 'Posted %B %d, %Y.').year         # Guardamos en una nueva columna, las fechas con formato valido (poseen anio).
    except ValueError:
       dfResultadoT.loc[dfResultadoT['posted'] == fila, 'nueva_fecha'] = None         # Si la fecha no es valida (no posee anio), guardamos un nan en su lugar.
       
dfResultadoT = dfResultadoT.dropna(subset=['nueva_fecha'])        #Eliminanos los datos nan de la columna de fechas.


for fila in dfResultadoT['review']:
    text_blob = TextBlob(fila)

    polarity = text_blob.polarity           # Calculamos la polaridad de las reviews.
    subjectivity = text_blob.subjectivity           # Calculamos la subjetividad de las rewiews.
        
    if subjectivity > 0.4:          # Si la subjetividad es mayor a 0.4, lo tomamos como una review valida [...] 
        if polarity > 0:
            dfResultadoT.loc[dfResultadoT['review'] == fila, 'positi'] = 1          # Si la polaridad es mayor a cero, es positiva.
        elif polarity < 0:
            dfResultadoT.loc[dfResultadoT['review'] == fila, 'negati'] = 1          # Si la polaridad es menor a cero, es negativa.
        else:
            dfResultadoT.loc[dfResultadoT['review'] == fila, 'neutro'] = 1          # Si la polaridad es cero, es neutra.
    else:
        dfResultadoT.loc[dfResultadoT['review'] == fila, 'neutro'] = 1          # [...] De lo contrario lo marcamos como neutro.
        
dfResultadoT = dfResultadoT.groupby(["recommend", "nueva_fecha", "item_id"]).sum().reset_index()            # Agrupamos los datos por recomendacion, fecha e item. Sumamos los resultados del analisis.
dfResultadoT = dfResultadoT.drop(['user_id', 'posted', 'review'], axis=1)         # Eliminamos la columnas que no utilizaremos.
dfResultadoT['nueva_fecha'] = dfResultadoT['nueva_fecha'].astype('int')         # Convertimos los años a integer.
dfResultadoT['neutro'] = dfResultadoT['neutro'].astype('int')         # Convertimos a integer.
dfResultadoT['positi'] = dfResultadoT['positi'].astype('int')         # Convertimos a integer.
dfResultadoT['negati'] = dfResultadoT['negati'].astype('int')         # Convertimos a integer.

### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok=True)
dfResultadoT.to_csv('Resultados/TConsulta.csv')

In [ ]:
dfResultadoT.head()

In [ ]:
#######################
### QUINTA CONSULTA ###
#######################

dfResultadoQ = pd.merge(dfGames, dfResultadoT, how = 'inner', left_on='id', right_on = 'item_id')            # Creamos un nuevo dataframe, agregando a la consulta utilizada para el punto anterior la fecha de lanzamiento de los juegos.
dfResultadoQ = dfResultadoQ.drop(['genres', 'item_id', 'app_name', 'tags', 'id', 'recommend', 'nueva_fecha'], axis = 1)           # Eliminamos columnas que no son necesarias.
dfResultadoQ = dfResultadoQ.groupby('release_date').sum().reset_index()         # Agrupamos por año y sumamos los analisis.
dfResultadoQ['release_date'] = dfResultadoQ['release_date'].astype('int')           # Convertimos los años en formato númerico.

### EXPORTAMOS A .CSV EL DATAFRAME ###
os.makedirs('Resultados', exist_ok = True)
dfResultadoQ.to_csv('Resultados/QConsulta.csv')

In [ ]:
dfResultadoQ.head()